# Stochastisches Bilevel-Modell für PV-Investitions- und Betriebsentscheidungen

Dieses Notebook implementiert ein stochastisches Bilevel-Modell, das eine Investitionsentscheidung für eine Photovoltaik-(PV)-Anlage und dazugehörige operative Entscheidungen (Netzbezug und Einspeisung) integriert. Der Code nutzt Julia, JuMP und Ipopt und bildet das in der zugehörigen PDF beschriebene Modell ab.

In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using JuMP, Ipopt, MathOptInterface
const MOI = MathOptInterface

  Activating project at `c:\Users\chrii\PycharmProjects\AMOProject`


MathOptInterface

In [3]:
Pkg.add("MathOptInterface")
Pkg.add("JuMP")
Pkg.add("Distributions")
Pkg.add("HiGHS")
Pkg.add("Clustering")
Pkg.add("Distances")
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("Ipopt")
Pkg.add("BilevelJuMP")

   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.toml`
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.toml`
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.toml`
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.toml`
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.toml`
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chrii\PycharmProjects\AMOProject\Project.

## Parameterdefinitionen

Hier werden die grundlegenden Parameter definiert, wie etwa die Anzahl der Zeitschritte, Investitionskosten, Strompreise sowie Parameter zur Modellierung der PV-Leistung und Emissionen.

In [4]:
# Anzahl der Zeitperioden (z.B. 12 Monate * 24 Stunden)
T = 288

# Anzahl der Szenarien pro Zeitschritt
num_scenarios = 5

# Investitionskosten (Beispielwert)
C_inv = 10 * 12

# Social Cost of Carbon ($/t CO2)
SCC = 50

# Kosten für Netzbezug und Erlös aus Einspeisung ($/kWh)
c_grid_buy = 0.25
c_grid_feed = 0.10

# Emissionsintensität (tCO2/kWh)
e_grid = 0.5

# PV-Parameter
η_0 = 0.18      # Nominale PV-Wirkungsgrad
β = 0.004       # Temperaturkoeffizient pro °C
T_ref = 25      # Referenztemperatur in °C
A = 1           # Fläche der PV-Anlage (Einheitlich)

1

## Definition der Effizienzfunktion

Die Funktion `efficiency` berechnet den temperaturabhängigen PV-Wirkungsgrad.

In [5]:
function efficiency(T_t)
    return η_0 * (1 - β * (T_t - T_ref))
end

efficiency (generic function with 1 method)

## Erzeugung der stochastischen Verteilungen

In diesem Abschnitt wird für jeden Zeitschritt ein Vektor von Verteilungen erstellt, der Solarstrahlung, Temperatur und Verbrauch abbildet. Dabei werden saisonale und tageszeitliche Effekte berücksichtigt.

In [6]:
using Distributions, Random

# Funktion zur Erzeugung eines Verteilungsvektors für einen Zeitschritt
function generate_distributions(si::Float64, consumption_mean::Float64, temp_mean::Float64, hour::Int)
    if hour < 6 || hour > 22
        # Nachtstunden: nahezu keine Sonneneinstrahlung
        solar = Beta(1, 1000)  
        temperature = Normal(temp_mean - 4, 9)
        consumption = Normal(consumption_mean/(consumption_mean - 1), consumption_mean / 11)
    else
        # Tagesstunden: solare Einstrahlung mit Beta-Verteilung
        β_param = ((si * (1 + si) / (si / 10)) - 1) * (1 - si)
        α_param = si * β_param / (1 - si)
        solar = Beta(α_param, β_param)
        temperature = Normal(temp_mean, 9)
        consumption = Normal(consumption_mean, consumption_mean / 11)
    end
    return [solar, consumption, temperature]
end

# Initialisierung des Verteilungsvektors für jeden Zeitschritt
dist_vec = Vector{Vector{Distribution}}(undef, 12 * 24)

for month in 1:12
    for hour in 1:24
        # Saisonale und tageszeitabhängige Parameter
        si = 0.25 + 0.1 * sin(2π * month / 12)  
        si = hour < 6 || hour > 22 ? 0.0 : si  
        consumption_mean = 90.0 + 10 * sin(2π * hour / 24)
        temp_mean = 13.0 + 5 * cos(2π * month / 12)
        
        # Speichern des Verteilungsvektors
        dist_vec[(month - 1) * 24 + hour] = generate_distributions(si, consumption_mean, temp_mean, hour)
    end
end

## Erzeugung der Dummy-Szenarien

Für jeden Zeitschritt werden 5 Szenarien für Solarstrahlung, Verbrauch und Temperatur erzeugt. Außerdem werden die Szenario-Wahrscheinlichkeiten (gleichverteilt) festgelegt.

In [7]:
# Gesamtzahl der Zeitschritte
T_total = 12 * 24

# Erzeugen der Dummy-Szenarien für Solar (Index 1 im Verteilungsvektor)
S_t_omega = [ [rand(dist_vec[t][1]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Erzeugen der Dummy-Szenarien für Verbrauch (Index 2 im Verteilungsvektor)
L_t_omega = [ [rand(dist_vec[t][2]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Erzeugen der Dummy-Szenarien für Temperatur (Index 3 im Verteilungsvektor)
T_t_omega = [ [rand(dist_vec[t][3]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Szenario-Wahrscheinlichkeiten (uniform verteilt)
Φ_t_omega = [ [1/num_scenarios for ω in 1:num_scenarios] for t in 1:T_total ]

288-element Vector{Vector{Float64}}:
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 ⋮
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]

## Testausgabe für den ersten Zeitschritt

Wir geben die für den ersten Zeitschritt (erste Stunde im Januar) generierten Werte aus.

In [8]:
println("Erste Stunde im Januar:")
println("Solar: ", S_t_omega[1])
println("Verbrauch: ", L_t_omega[1])
println("Temperatur: ", T_t_omega[1])
println("Szenario-Wahrscheinlichkeiten: ", Φ_t_omega[1])

Erste Stunde im Januar:
Solar: [0.0018371241305232757, 0.0019222050116626012, 0.0006121049730113572, 0.0015509414833692428, 0.0011960235735226493]
Verbrauch: [-4.555051351793297, 6.906169033407017, -3.0787691417663856, 5.44960305014942, -4.623582590774209]
Temperatur: [5.090044311020174, 10.465320724925686, 10.20987277504655, 36.258766571859034, 20.76415193462609]
Szenario-Wahrscheinlichkeiten: [0.2, 0.2, 0.2, 0.2, 0.2]


## Erstellung und Lösung des Optimierungsmodells

Das Optimierungsmodell wird in zwei Ebenen aufgeteilt:

- **Obere Ebene:** Investitionsentscheidung (Variable y_PV) unter Berücksichtigung der jährlichen Investitionskosten.
- **Untere Ebene:** Operative Entscheidungen (Netzbezug G_in und Einspeisung G_out) für jeden Zeitschritt und jedes Szenario. Die KKT-Bedingungen der unteren Ebene werden verwendet, um das bilevel-Problem in ein Single-Level-Problem zu transformieren.

Anhand der Energiebilanz wird sichergestellt, dass der Verbrauch durch PV-Erzeugung, Netzbezug und Einspeisung gedeckt wird.

In [9]:
model = Model(Ipopt.Optimizer)

# Obere Ebene: Investitionsentscheidung (Leader)
@variable(model, 0 <= y_PV <= 30)

# Untere Ebene: Operationelle Variablen (Follower)
@variable(model, G_in[1:T, 1:num_scenarios] >= 0)   # Netzimport
@variable(model, G_out[1:T, 1:num_scenarios] >= 0)   # Netzexport

# Dualvariablen für KKT-Bedingungen
@variable(model, λ[1:T, 1:num_scenarios])
@variable(model, nu_in[1:T, 1:num_scenarios] >= 0)
@variable(model, nu_out[1:T, 1:num_scenarios] >= 0)

for t in 1:T, ω in 1:num_scenarios
    # Berechnung der PV-Erzeugung als fester Wert unter Nutzung der Effizienzfunktion
    P_PV_val = efficiency(T_t_omega[t][ω]) * A * S_t_omega[t][ω]
    
    # Primal-Nebenbedingung: Energiebilanz
    @constraint(model, L_t_omega[t][ω] == y_PV * P_PV_val + G_in[t, ω] - G_out[t, ω])
    
    # Stationaritätsbedingungen (Gradienten der Lagrangefunktion)
    @constraint(model, c_grid_buy - λ[t, ω] + nu_in[t, ω] == 0)
    @constraint(model, -c_grid_feed + λ[t, ω] + nu_out[t, ω] == 0)
    
    # Komplementaritätsbedingungen (als nichtlineare Nebenbedingungen)
    @NLconstraint(model, nu_in[t, ω] * G_in[t, ω] == 0)
    @NLconstraint(model, nu_out[t, ω] * G_out[t, ω] == 0)
end

# Zielfunktion: Minimierung der Investitionskosten plus der gewichteten Betriebs- und Emissionskosten
@objective(model, Min,
    C_inv * y_PV +
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω] + SCC * e_grid * G_in[t, ω])
        for t in 1:T, ω in 1:num_scenarios)
)

optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    15840
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     8640

Total number of variables............................:     7201
                     variables with only lower bounds:     5760
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:     7200
Total number of inequality c

## Ausgabe der optimalen PV-Kapazität

Nach der Optimierung wird die optimale Investitionsentscheidung (PV-Kapazität) ausgegeben.

In [10]:
println("Optimale PV-Kapazität: ", value(y_PV))

Optimale PV-Kapazität: 22.992192988385213
